### Multiple Documents

In [1]:
import PyPDF2 as pdf
from PyPDF2 import PdfReader
import os
import pandas as pd
import re
import openai
import time
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
api_key_file = open("OPENAI_API_KEY", "r")
openai.api_key = api_key_file.read()

In [54]:
pdf_list = os.listdir("pdfs")

#### Assign ID

In [55]:
id_dict = dict(list(zip(pdf_list,range(0,len(pdf_list)))))
pdf_dict = {v: k for k, v in id_dict.items()}
pdf_dict

{0: 'Wi-Fi-6-OFDMA-App-Notes-091319-web.pdf',
 1: 'STBC.pdf',
 2: 'How to change the country code for WiFi.pdf',
 3: 'SPF53_SPF61_programmer guide.pdf',
 4: 'usb3-frequency-interference-papers.pdf',
 5: 'FCC-Rules-and-Regulations.pdf',
 6: 'programming uci shrinked.pdf',
 7: 'nor_nand_flash_guide.pdf',
 8: 'RadioChannelDFS.pdf'}

#### Iterate through

In [4]:
# Create dataframe for each individual pdf
df_list = []
for i,pdf in enumerate(pdf_list):
    
    cur_pdf = []
    
    with open("pdfs/" + pdf,"rb") as file:
        reader = PdfReader(file)
        for j in range(0, len(reader.pages)):
            cur_string = reader.pages[j].extract_text()
            cur_pdf.append("|start of page {} ".format(j+1)+cur_string + " end of page {}|".format(j+1))

    cur_df = pd.DataFrame({"page":list(range(1,len(cur_pdf)+1)),"text":cur_pdf})
    df_list.append((pdf,cur_df))

    

    


incorrect startxref pointer(1)


In [5]:
# Create embeddings for each page in each pdf
from openai.embeddings_utils import get_embedding
def embed(x):
    return get_embedding(x, engine="text-embedding-ada-002")



In [ ]:
for i in df_list:
    i[1]["pg_embedding"] = i[1]["text"].apply(lambda x: embed(x))

In [31]:
# Save all embeddings
for df_pair in df_list:
    np.save("Multiple Documents/embeddings/"+df_pair[0][0:-4]+"_ebded",np.array(list(df_pair[1]['pg_embedding'])))

In [33]:
def add_document(pdf):
    cur_id = max(id_dict.values()) + 1
    id_dict[pdf] = cur_id
    pdf_dict[cur_id] = pdf

    
    # Create dataframe for each individual pdf

    cur_pdf = []

    with open("pdfs/" + pdf,"rb") as file:
        reader = PdfReader(file)
        for j in range(0, len(reader.pages)):
            cur_string = reader.pages[j].extract_text()
            cur_pdf.append("|start of page {} ".format(j+1)+cur_string + " end of page {}|".format(j+1))

    cur_df = pd.DataFrame({"page":list(range(1,len(cur_pdf)+1)),"text":cur_pdf})
    cur_df["pg_embedding"] = cur_df["text"].apply(lambda x: embed(x))
    df_list.append((pdf,cur_df))
    for df_pair in df_list:
        np.save("Multiple Documents/embeddings/"+df_pair[0][0:-4]+"_ebded",np.array(list(df_pair[1]['pg_embedding'])))

In [34]:
add_document("Chess_ The Complete Guide To Chess - Master_ Chess Tactics, Chess Openings, and Chess Strategies.pdf")

In [8]:
# Load all embeddings

for df_pair in df_list:
    print(df_pair[0])
    cur_array = np.load("Multiple Documents/embeddings/"+df_pair[0][0:-4]+"_ebded.npy")
    df_pair[1]["pg_embedding"] = list(cur_array)


Wi-Fi-6-OFDMA-App-Notes-091319-web.pdf
STBC.pdf
How to change the country code for WiFi.pdf
SPF53_SPF61_programmer guide.pdf
usb3-frequency-interference-papers.pdf
FCC-Rules-and-Regulations.pdf
programming uci shrinked.pdf
nor_nand_flash_guide.pdf
RadioChannelDFS.pdf


In [39]:
# Average Embeddings for Individual Documents
avg_embedding_pairs = []

for df_pair in df_list:
    avg_embedding = np.array(list(df_pair[1]["pg_embedding"])).mean(axis=0)
    avg_embedding_pairs.append((df_pair[0],avg_embedding))


In [40]:
avg_embedding_pairs_df = pd.DataFrame({"pdf":[i[0] for i in avg_embedding_pairs], "avg_embedding":[i[1] for i in avg_embedding_pairs], "id":list(range(0,len(avg_embedding_pairs)))})

In [41]:
avg_embedding_pairs_df.to_csv("Multiple Documents/avg_embedding_pairs_df")


In [42]:
entire_df = avg_embedding_pairs_df.copy()



for i,df_pair in enumerate(df_list):
    df_pair[1]["id"] = np.full(df_pair[1].shape[0],i)
    
    

In [43]:
final_df = pd.concat([i[1] for i in df_list]).set_index('id').join(entire_df.set_index('id'))


In [44]:
final_df.to_csv("Multiple Documents/final_pdf_df")

In [46]:
def get_suitable_documents(prompt, k_docs=3):
    embedded_prompt = embed(prompt)
    ranking = avg_embedding_pairs_df.copy()
    ranking["cos"] = ranking["avg_embedding"].apply(lambda x: x@embedded_prompt)
    ranked_df = ranking.sort_values(by="cos",ascending=False)
    with open('Multiple Documents/logfile.txt', 'a') as f:
        f.write("looking at:"+str(list(ranked_df["pdf"])[0:k_docs]))
        f.write("\n")
    print("looking at:",list(ranked_df["pdf"])[0:k_docs])
    return list(ranked_df["id"][0:k_docs])

In [47]:
def get_page_text(pdf_ID,pg):
    try:
        text = final_df[final_df["page"] == pg].loc[pdf_ID]["text"]
        pdf_name = pdf_dict[pdf_ID]
        return "|||From the manual " + pdf_name + ": " + text
    except:
        return ""
    


In [48]:
def get_suitable_text(prompt,k_docs=8, k_pages=10):
    #Find suitable manuals
    embedded_prompt = embed(prompt)
    doc_ids = get_suitable_documents(prompt,k_docs)
    filtered_df = final_df[final_df.index.isin(doc_ids)]
    #Merge Documents
    filtered_df["pg_cos"] = filtered_df["pg_embedding"].apply(lambda x: np.array(x)@embedded_prompt)
    ranked_filtered_df = filtered_df.sort_values(by="pg_cos",ascending=False)
    #display(ranked_filtered_df.head())
    #Find suitable pages
    #rank_filtered_df["text"][0:k_pages]
    
    idx_pg_pairs = sorted(list(zip(ranked_filtered_df.index[0:k_pages],ranked_filtered_df['page'][0:k_pages])))
    idx_pg_dict = {}
    for key, value in idx_pg_pairs:
        idx_pg_dict.setdefault(key, []).append(value)
        
    for key,value in idx_pg_dict.items():
        page_list = value
        new_list = []
        for i,page in enumerate(page_list):
            if i == len(page_list)-1:
                new_list.append(page)
                new_list.append(page+1)
                break
            if (page_list[i+1] - page) >= 4:
                new_list.append(page-1)
                new_list.append(page)
                new_list.append(page+1)
            else:
                new_list.append(page-1)
                for pgno in list(range(page,page_list[i+1])):
                    new_list.append(pgno)
        idx_pg_dict[key] = list(set([i for i in new_list if i != 0]))
    for ID,pages in idx_pg_dict.items():
        with open('Multiple Documents/logfile.txt', 'a') as f:
            f.write("pages for {}:".format(pdf_dict[ID])+ str(pages))
            f.write("\n")
        print("pages for {}:".format(pdf_dict[ID]),pages)
    
    final_list = []
    for ID, pages in idx_pg_dict.items():
        for page in pages:
            final_list.append(get_page_text(ID,page))
    return " ".join(final_list)
    
    
    
    


In [18]:
from IPython.display import clear_output

In [50]:
premise = "You’re a technical support bot that has access to manuals. The manual information is here: "
messages=[
    {"role": "system", "content": premise}
      ]
response_list = []
def conversation(response):
    response_list.append(response)
# If there is no stored info, store the first 10 similar pages based on the user's prompts
    with open('Multiple Documents/logfile.txt', 'a') as f:
        f.write("---Query Start---\n")
        f.write(response + "\n")
    if len(messages) == 1:
        messages[0]["content"] += get_suitable_text(response,k_pages=4)

    #print(len(messages))
    
    content = "Question: " + response + " Notes: You CAN ONLY USE INFO FROM THE MANUAL TEXT I REPEAT.  You CAN ONLY USE INFO FROM THE MANUAL TEXT. If there is nothing available in the text, SAY I don't have the info. Do not generalize answers. Do not state the page numbers. Remember the answers can be from different manuals. Say 'According to my resources' if you understand this along with your response"
    messages.append({"role": "user", "content": content})
    
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k-0613",
    messages=messages,
    )
    message = completion["choices"][0]["message"]
    
    
    
    chat_response = completion.choices[0].message.content
    messages.append({"role": "assistant", "content": chat_response})
    for i in messages:
        if i['role'] == "user":
            pass
            #print("User: ", i['content'])
            #print("\n")
        elif i['role'] == "assistant":
            clear_output()
            with open('Multiple Documents/logfile.txt', 'a') as f:
                f.write(i['content'])
            print("ChatGPT: ", i['content'])
        #print("\n")
    with open('Multiple Documents/logfile.txt', 'a') as f:
        f.write("\n---Query End---\n\n\n")
    #print(messages[0]["content"])
    #return (response,messages[-1]["content"])
conversation("Give me tips for territorial domination in chess")




ChatGPT:  According to my resources, here are some tips for territorial domination in chess:

1. Control the center of the board: The center of the board is crucial in chess. Try to occupy and control the central squares (d4, d5, e4, e5) as early as possible. This will give you more options and flexibility to maneuver your pieces.

2. Push pawns to gain space: Advanced pawns create territorial advantage. By pushing your pawns forward, you can create more space for your pieces to operate and limit your opponent's options.

3. Keep your pieces coordinated and active: Develop and bring out your high-value pieces (queen, rooks, bishops, knights) and position them in a way that they support each other. Avoid leaving any piece inactive and utilize them effectively to control and dominate the board.

4. Defend your territories: Once you have gained control of certain areas, make sure to defend them adequately. Protect your pieces and maintain strong positions to prevent your opponent from inf

### Creating a Database and a Feature for Adding documents

In [137]:
conversation("Show me a step by step guide that is easy to follow to do this, and give easy explanations throughout")

ChatGPT:  I apologize, but I don't have access to step-by-step guides or specific instructions on modifying the C file to enable the PAPRD feature. The manual text does not provide detailed instructions on modifying the C file for this specific purpose. It only mentions that the PAPRD feature can be enabled at compile time by defining the "ATH_SUPPORT_PAPRD" compile flag.

However, I can provide some general guidance on how to modify C files:

1. Locate the C file: Find the specific C file that corresponds to the PAPRD feature implementation. The manual should provide the name or location of the file.

2. Open the C file: Use a text editor or an Integrated Development Environment (IDE) to open the C file.

3. Search for compile flags: Look for preprocessor directives or `#ifdef` statements that check for compile flags. These flags determine whether certain features or code blocks are included or excluded during compilation.

4. Add the compile flag: Depending on the existing code struc

In [95]:
final_df

,page,text,pg_embedding,pdf,avg_embedding
id,,,,,
0,1,"|start of page 1 © 2019 LitePoint, A Teradyne ...","[0.0035223152954131365, 0.022471480071544647, ...",Wi-Fi-6-OFDMA-App-Notes-091319-web.pdf,"[0.005741071686233309, 0.014976223549720917, -..."
0,2,|start of page 2 Wi-Fi 6 OFDMA Pre-correction ...,"[0.018090346828103065, 0.018831972032785416, -...",Wi-Fi-6-OFDMA-App-Notes-091319-web.pdf,"[0.005741071686233309, 0.014976223549720917, -..."
0,3,|start of page 3 Wi-Fi 6 OFDMA Pre-correction ...,"[0.006059110164642334, 0.012037656269967556, -...",Wi-Fi-6-OFDMA-App-Notes-091319-web.pdf,"[0.005741071686233309, 0.014976223549720917, -..."
0,4,|start of page 4 Wi-Fi 6 OFDMA Pre-correction ...,"[0.008734132163226604, 0.007961680181324482, -...",Wi-Fi-6-OFDMA-App-Notes-091319-web.pdf,"[0.005741071686233309, 0.014976223549720917, -..."
0,5,|start of page 5 Wi-Fi 6 OFDMA Pre-correction ...,"[0.023909470066428185, 0.008188648149371147, -...",Wi-Fi-6-OFDMA-App-Notes-091319-web.pdf,"[0.005741071686233309, 0.014976223549720917, -..."
...,...,...,...,...,...
8,2,|start of page 2 Dynamic Frequency Selec...,"[-0.012744361534714699, 0.010036693885922432, ...",RadioChannelDFS.pdf,"[-0.0008843569473053018, 0.008315470098750666,..."
8,3,|start of page 3 Dynamic Frequency Selec...,"[-0.003540026256814599, 0.009645288810133934, ...",RadioChannelDFS.pdf,"[-0.0008843569473053018, 0.008315470098750666,..."
8,4,|start of page 4 Dynamic Frequency Selec...,"[-0.00892565492540598, 0.01575360633432865, 0....",RadioChannelDFS.pdf,"[-0.0008843569473053018, 0.008315470098750666,..."


In [28]:

#pdf_dict

8

In [30]:
df_list[0]

('Wi-Fi-6-OFDMA-App-Notes-091319-web.pdf',
     page                                               text  \
 0      1  |start of page 1 © 2019 LitePoint, A Teradyne ...   
 1      2  |start of page 2 Wi-Fi 6 OFDMA Pre-correction ...   
 2      3  |start of page 3 Wi-Fi 6 OFDMA Pre-correction ...   
 3      4  |start of page 4 Wi-Fi 6 OFDMA Pre-correction ...   
 4      5  |start of page 5 Wi-Fi 6 OFDMA Pre-correction ...   
 5      6  |start of page 6 Wi-Fi 6 OFDMA Pre-correction ...   
 6      7  |start of page 7 Wi-Fi 6 OFDMA Pre-correction ...   
 7      8  |start of page 8 Wi-Fi 6 OFDMA Pre-correction ...   
 8      9  |start of page 9 Wi-Fi 6 OFDMA Pre-correction ...   
 9     10  |start of page 10 Wi-Fi 6 OFDMA Pre-correction...   
 10    11  |start of page 11 TRADEMARKS\nLitePoint and th...   
 
                                          pg_embedding  id  
 0   [0.0035223152954131365, 0.022471480071544647, ...   0  
 1   [0.018090346828103065, 0.018831972032785416, -...   0  
 2  

In [65]:
np.array(embed("hi")).shape

(1536,)

### Reformatting Everything

In [66]:
from docarray import BaseDoc
from docarray.typing import NdArray

class PageDoc(BaseDoc):
    page_txt: str = ''
    pdf_id: int = 0
    page_no: int = 0
    embedding: NdArray[1536]

class ManualDoc(BaseDoc):
    pdf_title: str = ''
    pdf_id: int = 0
    embedding: NdArray[1536]

class ToyDoc(BaseDoc):
    text: str = ''
    embedding: NdArray[128]

In [67]:
from docarray import DocList
from vectordb import InMemoryExactNNVectorDB, HNSWVectorDB

In [91]:
?InMemoryExactNNVectorDB

In [70]:
db = InMemoryExactNNVectorDB[ToyDoc](workspace='./prototype')

# Index a list of documents with random embeddings
doc_list = [ManualDoc(pdf_title=f'toy doc {i}', pdf_id = i, embedding=np.random.rand(1536)) for i in range(1000)]
db.index(inputs=DocList[ManualDoc](doc_list))

# Perform a search query
# query = ToyDoc(text='query', embedding=np.random.rand(128))
# results = db.search(inputs=DocList[ToyDoc]([query]), limit=10)

# # Print out the matches
# for m in results[0].matches:
#     print(m)

TypeError: <lambda>() missing 3 required positional arguments: 'is_singleton_doc', 'parameters_is_pydantic_model', and 'parameters_model'

In [ ]:
# A way to detect files in local and see if one should be added

# Vector DB with embeddings for all manuals
# Vector DB for pages for each specific manuals

# Embedding thresholding
# Custom prompts and continuity between conversation
# Way to add more context if needed